# Interactive plotting with holoviews

## Authors & Contributors

### Authors

- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)

### Contributors

- Alejandro Coca-Castro, The Alan Turing Institute (United Kingdom), [@acocac](https://github.com/acocac)

<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> Overview
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>How to clip Xarray DataSet with a shapefile to plot an area of interest?</li>
        <li>How to generate interactive maps with Holoviews?</li>
        <li>How to browse multiple times with Holoviews interactive maps?</li>
        <li>How to generate an interactive 1D plot with Holoviews?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about Holoviews and Xarray for creating interactive plots</li>
        <li>Learn to create interactive maps with Holoviews</li>
        <li>Learn to create multiple interactive plots with holoviews</li>
        <li>Learn to create a 1D line plot with holoviews</li>
    </ul>
</div>

## Context


We will be using [holoviews](https://holoviews.org/) with Xarray to visualize the Vegetation Condition Index (VCI) ([1] Kogan, 1995), a well-established indicator to estimate droughts from remote sensing data.

### Data

This episode we will use data that has been generated in the previous episode.

If the dataset is not present in the same folder as this Jupyter notebook, it will be downloaded from zenodo using `pooch`, a very handy python-based library to download and cache your data files locally (see further info [here](https://www.fatiando.org/pooch/latest/index.html)).

In [ ]:
import pooch

cgls_file = pooch.retrieve(
    url="https://zenodo.org/record/6969999/files/C_GLS_NDVI_20220101_20220701_Lombardia_S3_2_masked.nc",
    known_hash="md5:be3f16913ebbdb4e7af227f971007b22",
    path=f".",
)

## Setup

This episode uses the following Python packages:

- xarray
- rioxarray
- netcdf4
- h5netcdf
- wget
- numpy
- matplotlib
- cartopy
- hvplot
- geopandas

Please install these packages if not already available in your Python environment. Below, we only install packages that are not available in the EGI-ACE EOSC deployment of Pangeo for the FOSS4G course.

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

In [ ]:
import xarray as xr

## Open local dataset

In [ ]:
cgls_ds = xr.open_dataset(cgls_file)

:::{tip}
If you get an error with the previous command, check the previous episode where the input file CAMS-PM2_5-20211222.netcdf is downloaded locally and it is in the same directory as your Jupyter Notebook.
:::

In [ ]:
cgls_ds

## Clipping data according to a polygon

One of the basic concepts in GIS is to clip data using a vector geometry. Xarray is not directly capable of dealing with vectors but thanks to rioxarray that can be easily achieved.
Rioxarray extends Xarray with most of the features that Rasterio (GDAL) brings.

## Read a shapefile with the Area Of Interest (AOI)

In [ ]:
import geopandas as gpd

We define the area of interest using the Global Administrative Unit Layers [GAUL G2015_2014](https://data.apps.fao.org/map/catalog/srv/eng/catalog.search#/metadata/9c35ba10-5649-41c8-bdfc-eb78e9e65654) provided by FAO-UN (see [Documentation](https://data.apps.fao.org/map/catalog/srv/api/records/9c35ba10-5649-41c8-bdfc-eb78e9e65654/attachments/GAUL2015_Documentation.zip)).
[`GeoPandas`](https://geopandas.org/en/stable/), a python-based library extending the capabilities of [`Pandas`](https://pandas.pydata.org/) to deal with geometry and spatial operations, will help to manage geodata.

The official data distribution from FAO is through the WFS service (below how to retrieve data):

```
GAUL = gpd.read_file('https://data.apps.fao.org/map/gsrv/gsrv1/gaul/wfs?'
                     'service=WFS&version=2.0.0&'
                     'Request=GetFeature&'
                     'TypeNames=gaul:g2015_2014_2&'
                     'srsName=EPSG%3A4326&'
                     'maxFeatures=2500&'
                     'outputFormat=json')
```
Unfortunately it seems that the service is pretty slow. As an alternative to this approach the JRC MARS unit is distributing the original dataset that was in shapefile format. To accelerate the fetch we highly recommend to follow this approach.

For the training course, we also created a tiny file containing information about Italy only.

In [ ]:
try:
    GAUL = gpd.read_file('Italy.geojson')
except:
    GAUL = gpd.read_file('zip+https://mars.jrc.ec.europa.eu/asap/files/gaul1_asap.zip') 

Data are organized in a tabular structure. For each element an index, data (made of columns) and a geometry are defined.

Geometries are defined through [shapely](https://shapely.readthedocs.io/en/stable/) geometry objects with three different basic classes:

- Points and Multi-Points
- Lines and Multi-Lines
- Polygons and Multi-Polygons

In [ ]:
GAUL.head(5)

In the cell below, we subset the polygon geometry in which the `name1` field equals to `Lombardia`.

In [ ]:
AOI_name = 'Lombardia'
AOI = GAUL[GAUL.name1 == AOI_name]
AOI_poly = AOI.geometry
AOI_poly

In a second step this geometry has to be defined is the EPSG:4326 Geodetic coordinate reference system.

In [ ]:
cgls_ds = cgls_ds.NDVI.rio.write_crs(4326)

Once this has been done we can clip the data with the polygon that has been obtained through geopandas at the beginning of the notebook.

In [ ]:
NDVI_AOI = cgls_ds.rio.clip(AOI_poly, crs=4326)

## Visualize with matplotlib

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
fig = plt.figure(1, figsize=[20, 10])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
#ax.set_extent([15.5, 27.5, 36, 41], crs=ccrs.PlateCarree()) # lon1 lon2 lat1 lat2
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

NDVI_AOI.sel(time='2022-06-01').plot(ax=ax, transform=ccrs.PlateCarree(), cmap="RdYlGn")

# One way to customize your title
plt.title("S3 NDVI over Lombardia", fontsize=18)

## Visualization with holoviews

In [ ]:
import holoviews as hv
import hvplot.xarray
import hvplot.pandas

Plotting data through the HoloViews back-end thanks to the hvplot that acts as high-level plotting API.

In [ ]:
NDVI_AOI.isel(time=0).hvplot(cmap="RdYlGn", width=1000, height=1000)

Having a look to data distribution can reveal a lot about the data.

In [ ]:
NDVI_AOI[0].hvplot.hist(cmap="RdYlGn",bins=25, width=800, height=700)

### Multi-plots using groupby

To be able to visualize interactively all the different available times, we can use `groupby` time.

In [ ]:
NDVI_AOI.hvplot(groupby ='time', cmap="RdYlGn", width=800, height=700)

Once data is limited to the AOI a histogram can be added to the visualization.

In [ ]:
NDVI_AOI.hvplot(groupby='time', cmap='RdYlGn', width=800, height=700 ).hist()

### Plot a single point over the time dimension

In [ ]:
NDVI_AOI.sel(lat=45.88, lon=8.63, method='nearest').hvplot(ylim=(-0.08, 0.92))

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>rioarray for clipping over an area of interest</li>
        <li>Interactive plot with Holoviews and Xarray</li>
    </ul>
</div>